In [1]:
import numpy as np
import itertools
import datetime

import tensorflow as tf
import sklearn as sk

from gensim.models import Word2Vec

import Functions as fn
import Iterator as it
from DS import DS
from Set import pool
from FFModel import FF_Model
from RNNModel import RNN_Model

C:\Users\Luka\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Data Loading

In [2]:
Dataset = pool()
Dataset.load_texts('raw_texts')
Dataset.load_labels('raw_labels')

Raw Text Load Complete
Raw Labels Load Complete


In [3]:
target_dict = fn.load_labels('labels')

Label Load Complete


# Embedding Generation

In [4]:
#Dataset.process_for_embedding()
#sentences = Dataset.get_sentences()
#fn.write_sentences('sentences')
sentences = fn.load_sentences('sentences')

#model = Word2Vec(sentences, min_count=1, size=100)
#model.save('W2V')
model = Word2Vec.load('W2V')

vocab = model.wv.vocab.keys()

Sentence Load Complete


# Layer and Index Loading

In [5]:
#word_indices, emb_layer = fn.get_index_and_emb_layer(model)
#fn.write_word_indices(word_indices, 'word_indices')
#fn.write_emb_layer(emb_layer, 'emb_layer')

word_indices = fn.load_word_indices('word_indices')
emb_layer = fn.load_emb_layer('emb_layer')

Word Indices Load Complete
Embedding Layer Load Complete


In [18]:
np.shape(model.wv.syn0)
test = np.vstack((np.zeros(100, dtype=np.float32), np.zeros(100, dtype=np.float32), model.wv.syn0))
#print(type(emb_layer[0,0]))
print(emb_layer[:3])
print(test[:3])

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00  

# RNN Testing

In [7]:
train_cases = pool((Dataset.get_DS(stage='test', labelled='yes')).data[:-25])
validation_cases = pool(Dataset.get_DS(stage='test', labelled='yes').data[-25:])
test_cases = Dataset.get_DS(stage='train', labelled='yes')

train_cases.process_for_s2s_testing()
validation_cases.process_for_s2s_testing()
test_cases.process_for_s2s_testing()

In [2]:
cell = tf.nn.rnn_cell.LSTMCell(100)
a, b = tf.contrib.legacy_seq2seq.embedding_attention_seq2seq([tf.constant([0])], [tf.constant([0])], cell, 1, 1, 5)
[print(v.name) for v in tf.trainable_variables() if "embedding" in v.name]

embedding_attention_seq2seq/rnn/embedding_wrapper/embedding:0
embedding_attention_seq2seq/rnn/embedding_wrapper/lstm_cell/kernel:0
embedding_attention_seq2seq/rnn/embedding_wrapper/lstm_cell/bias:0
embedding_attention_seq2seq/embedding_attention_decoder/embedding:0
embedding_attention_seq2seq/embedding_attention_decoder/attention_decoder/AttnW_0:0
embedding_attention_seq2seq/embedding_attention_decoder/attention_decoder/AttnV_0:0
embedding_attention_seq2seq/embedding_attention_decoder/attention_decoder/kernel:0
embedding_attention_seq2seq/embedding_attention_decoder/attention_decoder/bias:0
embedding_attention_seq2seq/embedding_attention_decoder/attention_decoder/output_projection_wrapper/lstm_cell/kernel:0
embedding_attention_seq2seq/embedding_attention_decoder/attention_decoder/output_projection_wrapper/lstm_cell/bias:0
embedding_attention_seq2seq/embedding_attention_decoder/attention_decoder/output_projection_wrapper/kernel:0
embedding_attention_seq2seq/embedding_attention_decoder/a

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [8]:
sets = {}
sets['train_set'], sets['train_labels'], _ = train_cases.get_s2s_sets(word_indices)
sets['validation_set'], sets['validation_labels'], _ = validation_cases.get_s2s_sets(word_indices)
sets['test_set'], sets['test_labels'], sets['test_words'] = test_cases.get_s2s_sets(word_indices)

In [174]:
import tensorflow as tf
import numpy as np
import warnings
import matplotlib.pyplot as plt
import sklearn as sk

from Iterator import RNNIterator


class S2S_Model:

    def __init__(self, enc_sym, dec_sym, emb_size, state_size, dropout=1.0, learn_rate=0.01, emb_layer=False):
        self.enc_sym = enc_sym
        self.dec_sym = dec_sym
        self.emb_size = emb_size
        self.state_size = state_size
        self.dropout = dropout
        self.learn_rate = learn_rate
        self.emb_layer = emb_layer
        self.graph = None
        self.sess = None

    def reset_graph(self):
        if self.sess:
            self.sess.close()
        tf.reset_default_graph()

    def build_graph(self):
        self.reset_graph()

        # Placeholders
        x = tf.placeholder(tf.int32, shape=[None, 61])  # [batch_size, num_steps]
        y = tf.placeholder(tf.int32, shape=[None, 61])
        seqlen = tf.placeholder(tf.int32)
        keep_prob = tf.placeholder(tf.float32)
        previous = tf.placeholder(tf.bool)
        
        x_in = tf.unstack(tf.transpose(x, perm=[1, 0]))
        y_in = tf.unstack(tf.transpose(y, perm=[1, 0]))
        
        # RNN
        cell = tf.nn.rnn_cell.LSTMCell(self.state_size)
        #cell = tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=keep_prob)
        s2s_outputs, final_state = tf.contrib.legacy_seq2seq.embedding_attention_seq2seq(encoder_inputs = x_in,
                                                                                         decoder_inputs = y_in,
                                                                                         cell = cell,
                                                                                         num_encoder_symbols=self.enc_sym,
                                                                                         num_decoder_symbols=self.dec_sym,
                                                                                         embedding_size=self.emb_size,
                                                                                         num_heads=1,
                                                                                         output_projection=None,
                                                                                         feed_previous=previous,
                                                                                         initial_state_attention=False)
        logits = tf.nn.softmax(s2s_outputs)
        preds = tf.arg_max(logits, 1)
        
        # Softmax layer
        #with tf.variable_scope('softmax'):
        #    W = tf.Variable(tf.truncated_normal(shape=[self.state_size, self.num_classes], stddev=0.05))
        #    b = tf.Variable(tf.constant(0.1, shape=[self.num_classes]))
        #logits = tf.matmul(last_rnn_output, W) + b
        #preds = tf.nn.softmax(logits)
        
        #prediction = tf.argmax(preds, 1)

        #loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
        #train_step = tf.train.AdamOptimizer(self.learn_rate).minimize(loss)

        self.graph = {'x': x,
                      'y': y,
                      'keep_prob': keep_prob,
                      'previous': previous,
                      's2s_outputs': s2s_outputs}

    def train(self, sets, epochs=10, batch=50, report_percentage=1, show_progress=False, show_plot=False):
        # Start a tf session and run the optimisation algorithm
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

        trainer = RNNIterator(sets['train_set'], np.argmax(sets['train_labels'], 1), sets['train_lengths'])

        train_truth = np.argmax(sets['train_labels'], 1)
        validation_truth = np.argmax(sets['validation_labels'], 1)
        test_truth = np.argmax(sets['test_labels'], 1)

        train_feed = {self.graph['x']: sets['train_set'],
                      self.graph['y']: np.argmax(sets['train_labels'], 1),
                      self.graph['seqlen']: sets['train_lengths'],
                      self.graph['keep_prob']: 1.0}
        validation_feed = {self.graph['x']: sets['validation_set'],
                           self.graph['y']: np.argmax(sets['validation_labels'], 1),
                           self.graph['seqlen']: sets['validation_lengths'],
                           self.graph['keep_prob']: 1.0}
        test_feed = {self.graph['x']: sets['test_set'],
                     self.graph['y']: np.argmax(sets['test_labels'], 1),
                     self.graph['seqlen']: sets['test_lengths'],
                     self.graph['keep_prob']: 1.0}

        train_f1_score = []
        validation_f1_score = []

        mark = (epochs * (len(sets['train_set']) // batch) * report_percentage) // 100
        check_point = []
        N = 0

        warnings.simplefilter("ignore")
        while trainer.epochs < epochs:
            trd, trl, trle = trainer.next_batch(batch)
            if N % mark == 0:
                prediction = self.sess.run(self.graph['prediction'], feed_dict=train_feed)
                train_f1_score.append(sk.metrics.f1_score(train_truth, prediction, pos_label=0))
                prediction = self.sess.run(self.graph['prediction'], feed_dict=validation_feed)
                validation_f1_score.append(sk.metrics.f1_score(validation_truth, prediction, pos_label=0))
                check_point.append(N)
                if show_progress: print("Progress: %d%%" % (N * report_percentage // mark), end="\r")
            feed = {self.graph['x']: trd, self.graph['y']: trl, self.graph['seqlen']: trle, self.graph['keep_prob']: self.dropout}
            self.sess.run(self.graph['ts'], feed_dict=feed)
            N += 1
        warnings.simplefilter("default")

        if show_plot:
            np_check_point = np.array(check_point)
            np_train_f1 = np.array(train_f1_score)
            np_val_f1 = np.array(validation_f1_score)

            plt.plot(np_check_point, np_train_f1, label="Train")
            plt.plot(np_check_point, np_val_f1, label="Validation")
            plt.plot(np_check_point, np.ones(len(np_check_point))*0.35, label="Baseline")
            plt.xlabel("Batches")
            plt.ylabel("F1-Score")
            plt.legend()
            plt.show()

        test_f1_score = sk.metrics.f1_score(test_truth, self.sess.run(self.graph['prediction'], feed_dict=test_feed),
                                            pos_label=0)
        if show_progress:
            print('FInal Values: Tr-F1: {:.4f}, Val-F1: {:.4f}'.format(train_f1_score[-1], validation_f1_score[-1]))
            print("Test F1-Score: {:.4f}\n".format(test_f1_score))
        return train_f1_score, validation_f1_score, test_f1_score

    def predict(self, data, seqlen):
        dummy = [1 for i in range(len(data))]
        feed = {self.graph['x']: data, self.graph['y']: dummy, self.graph['seqlen']: seqlen, self.graph['keep_prob']: 1.0}
        return self.sess.run(self.graph['prediction'], feed_dict=feed)

    def close(self):
        self.sess.close()

In [175]:
S2S = S2S_Model(enc_sym=len(word_indices),
                dec_sym=6,
                emb_size=len(emb_layer[0]),
                state_size=100,
                learn_rate=0.001,
                emb_layer=emb_layer)
S2S.build_graph()
#results_list = RNN.train(sets=sets, epochs=5, batch=50, report_percentage=1, show_progress=True, show_plot=True)

In [188]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    out = sess.run(S2S.graph['s2s_outputs'], feed_dict={S2S.graph['x']:[sets['test_set'][0]],
                                                        S2S.graph['y']:[sets['test_labels'][0]],
                                                        S2S.graph['keep_prob']:1.0,
                                                        S2S.graph['previous']:False})
    out = tf.Variable(out)
    print(tf.shape(out))
    #logits = tf.nn.softmax(out)
    #print(logits)
    #print(tf.arg_max(tf.nn.softmax(out, 1), 1).eval())

ValueError: Argument must be a dense tensor: [array([[-0.12008914,  0.21895055, -0.14263801,  0.03613903,  0.03949914,
        -0.05319323]], dtype=float32), array([[-0.13080209,  0.22006416, -0.12917237,  0.05272328,  0.04248069,
        -0.05863551]], dtype=float32), array([[-0.14260025,  0.22366118, -0.11931948,  0.0601925 ,  0.04219366,
        -0.06657192]], dtype=float32), array([[-0.15230566,  0.22749868, -0.11189627,  0.06419074,  0.04103793,
        -0.07333551]], dtype=float32), array([[-0.16017942,  0.23118873, -0.10617298,  0.06606901,  0.03940512,
        -0.07893278]], dtype=float32), array([[-0.16656819,  0.2345047 , -0.10166955,  0.06662822,  0.03754642,
        -0.0835712 ]], dtype=float32), array([[-0.17179909,  0.23735973, -0.09807751,  0.06634965,  0.03561754,
        -0.08747087]], dtype=float32), array([[-0.17613699,  0.2397463 , -0.09518957,  0.0655349 ,  0.03371098,
        -0.09080699]], dtype=float32), array([[-0.17977872,  0.24169622, -0.09286059,  0.06438202

In [165]:
with tf.Session() as sess:
    #a = tf.range(27)
    #a = tf.reshape(a, [3,3,3])
    a = tf.constant(sets['test_set'][0])
    print(a.eval())
    print(tf.shape(a).eval())
    b = np.array(a)
    print(b.eval())

[34606     0 37927 37489 39417 39984 10168 29731 24182  4838 24987 10835
  4838 23539 18428 16984 45872 11617 23539 15090 16984 45872 39417 20411
 23539 27866 16984 45872  6671 23539 45840 16984 45872  6671 23539 34606
 16984 45872 16360 39417 16984 23539 34061 16984 45872 39417 20411 23539
 25081 16984 45872 39417 30196  7961 16329 41798  9748 34228 34518 20392
 43350]
[61]


AttributeError: 'numpy.ndarray' object has no attribute 'eval'

In [168]:
maxl = 0
minl = float('inf')
for case in train_cases.data:
    for inp in case.test_text:
        maxl = max(maxl, len(inp))
        minl = min(minl, len(inp))

for case in validation_cases.data:
    for inp in case.test_text:
        maxl = max(maxl, len(inp))
        minl = min(minl, len(inp))
        
for case in test_cases.data:
    for inp in case.test_text:
        maxl = max(maxl, len(inp))
        minl = min(minl, len(inp))

print(maxl, minl)

97 11


In [ ]:
res = RNN.predict(sets['test_set'], sets['test_lengths'])
tru = np.argmax(sets['test_labels'], 1)

In [ ]:
token_perf = fn.token_perf(res, tru)

In [ ]:
baseline = [0 if word in target_dict['reasons'] else 1 for word in sets['test_words']]
sk.metrics.f1_score(tru, baseline, pos_label=0, average='binary')

In [ ]:
fn.category_words(sets['test_words'], res, tru, 1, 0)

In [ ]:
fn.colour_text(sets['test_words'], res, tru)

In [ ]:
testers = Dataset.get_DS(stage='train', labelled='yes')
phrase_perf = fn.phrase_perf(target, RNN, testers, word_indices, side_words=[lw, rw], tfpn=True, show_phrases=True, case_info=True, rnn=True)

# HyperParameter Scan

In [ ]:
emb_sizes = [100]
emb_models = 1
target_saturations = [0.05, 0.1, 0.2, 0.5, 0.7]
layer_sizes = [50]
dropouts = [1.0]
learn_rates = [0.01]
epochs = [100]
NN_num = 5

case_num = len(emb_sizes)*emb_models*len(layer_sizes)*len(target_saturations)*len(epochs)*len(dropouts)*len(learn_rates)*NN_num
print(case_num)

In [ ]:
max_performance = 0
n = 1

for emb_size in emb_sizes:
    print('Model Number: %d/%d' %(n, case_num))
    for i in range(emb_models):
        model = Word2Vec(sentences, min_count=1, size=emb_size)
        for saturation in target_saturations:
            sets = fn.get_traintest2 (labelled_cases, model)
            fn.saturate_training_set(sets, model, target_dict['medications'], saturation)
            for layer_size in layer_sizes:
                for drop in dropouts:
                    for rate in learn_rates:
                        for epoch in epochs:
                            for j in range(NN_num):
                                print('Model Number: %d/%d' %(n, case_num))
                                print('ES: %d EM: %d sat: %f, LS: %d, drop: %f, LR: %f, epochs: %d, NN: %d' \
                                       % (emb_size, i, saturation, layer_size, drop, rate, epoch, j))
                                NN = FF_Model(input_size=emb_size, layers=[layer_size], dropout=drop, learn_rate=rate)
                                NN.build_graph()
                                NN.train(sets, epochs=epoch)
                                res = NN.predict(sets['test_set'])
                                tru = np.argmax(sets['test_labels'], 1)
                                perf = sk.metrics.f1_score(tru, res, pos_label=0)
                                if perf > max_performance:
                                    max_performance = perf
                                    NN.save_model('gold')
                                    model.save('gold/GOLDEMB')
                                NN.close()
                                n += 1